In [106]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from os import mkdir
import imageio
from glob import glob
from tqdm import tnrange
from shutil import copyfile

In [86]:
class Video(object):

    def __init__(self, file_name):
        self.file_name = file_name
        self.cap = cv2.VideoCapture(self.file_name)

        # get number of frames, 7 is the propID for number of frames
        if self.file_name[-4:] == 'h264':
            try:
                meta_name = self.file_name[:-5] + '.npy'
                vshape = np.load(meta_name)
                self.num_frame = vshape[0]
            except Exception:
                self.num_frame = 0
                print('Could not find metatdata file '+meta_name)

        else:
            self.num_frame = self.cap.get(7)

    def read_frame(self, i):
        # 1 is the propID for current frame
        self.cap.set(1, i)
        ret, frame = self.cap.read()
        if ret == 1:
            return frame[:, :, 2::-1]
        else:
            raise Exception('Bad Frame')

    @property
    def total_frames(self):
        return self.num_frame

    def close(self):
        self.cap.release()
        
def find_bad_frame(data_set, likelihood=0.95, check_progress=False, progress_bar=None):
    data_set_name = data_set.keys()[0][0]
    num_label = data_set[data_set_name].keys().levshape[0]

    if check_progress:
        progress_bar.setValue(50)

    labels = list()
    for label_id in range(num_label):
        labels.append(data_set[data_set_name].keys()[label_id*3][0])

    label_bad_frame = data_set[data_set_name][labels[0]]['likelihood'] < likelihood
    for label in labels:
        label_bad_frame = ((data_set[data_set_name][label]['likelihood'] < likelihood) | label_bad_frame)

    bad_frames = data_set.index[label_bad_frame].tolist()
    return bad_frames

In [87]:
video_name = './videos/EE11_06_20180609_2Track.1'

def save_bad_frames(video_name,suffix='DeepCut_resnet101_trackingledsAug13shuffle1_500000'):
    video_file_name = video_name + '.mp4'
    dataset_file_name = video_name + suffix + '.h5'
    try:
        mkdir(video_name+'_bad_frames')
    except:
        print('Already exist!')
    video = Video(video_file_name)
    dataset = pd.read_hdf(dataset_file_name)
    bad_frame_ids = find_bad_frame(dataset)
    dataset_name = dataset.keys()[0][0]
    num_labels = dataset[dataset_name].keys().levshape[0]
    labels = list()
    for label_id in range(num_label):
        labels.append(dataset[dataset_name].keys()[label_id*3][0])

    xs = np.zeros((len(bad_frame_ids),num_labels))
    ys = np.zeros((len(bad_frame_ids),num_labels))

    for i, bad_frame_id in enumerate(bad_frame_ids):
        frame = video.read_frame(bad_frame_id)
        imageio.imwrite(video_name+'_bad_frames/img'+str(i)+'.png', frame)
    bad_dataset = np.zeros((len(bad_frame_ids),4))
    bad_dataset[:,0:2] = dataset.loc[bad_frame_ids].values[:,0:2]
    bad_dataset[:,2:4] = dataset.loc[bad_frame_ids].values[:,3:5]
    np.save(video_name+'_bad_frames/Results.npy',bad_dataset)

    video.close()

In [89]:
file_name_list = glob('./videos/*.mp4')
for i in tnrange(len(file_name_list)):
    file_name = file_name_list[i]
    video_name = file_name[:-4]
    save_bad_frames(video_name)

Already exist!



In [108]:
results = np.zeros((1,4))
counter = 900
try:
    mkdir('./videos/organized')
except:
    print('Already exist')
folder_list = glob('./videos/*_bad_frames')
for folder_name in folder_list:
    array_file_name = folder_name + '/Results.npy'
    result = np.load(array_file_name)
    for i in range(result.shape[0]):
        copyfile(folder_name +'/img'+str(i)+'.png','./videos/organized/img'+str(counter)+'.png')
        counter += 1
    results = np.concatenate((results,result))
    
results = results[1:,:]

Already exist


In [109]:
np.save('./videos/organized/Results.npy',results)

In [110]:
results_csv = results.reshape(results.shape[0]*2,-1)

In [112]:
results

array([[432.66251898, 261.94742393, 437.10376453, 256.71255517],
       [435.79728346, 258.51185024, 436.82915246, 256.16931295],
       [436.02340235, 259.48951316, 437.00540698, 256.5559299 ],
       ...,
       [410.74445844, 255.78184605, 405.80944407, 243.55514672],
       [449.62667489, 238.79143238, 450.33530486, 238.51092792],
       [450.07091057, 242.43125045, 450.74619281, 240.54321957]])

In [118]:
np.savetxt('./videos/organized/Results.csv',results_csv,fmt='%10.5f',delimiter=',')